In [ ]:
from textblob import TextBlob
import tweepy
import pandas as pd
import json
import re
import users_twitter
from credentials import twitter_credentials

In [ ]:
auth = tweepy.OAuthHandler(twitter_credentials.api_key, twitter_credentials.api_secret_key)
auth.set_access_token(twitter_credentials.access_token, twitter_credentials.access_token_secret)
api = tweepy.API(auth)
df_depression_users = pd.DataFrame()
df_non_depression_users = pd.DataFrame()
df_tweets = pd.DataFrame()

In [ ]:
def extract_tweets(user_timeline, user_api):
    tweets = []
    global df_tweets
    for i in user_timeline.items():
        tweet_id = i.id_str
        tweet_text = i.text
        tweet_rts = i.retweet_count
        tweet_favs = i.favorite_count
        tweet_date = i.created_at
        tweet_lang = ''
        tweet = {
            'id': tweet_id,
            'text': tweet_text,
            'number_of_retweets': tweet_rts,
            'number_of_favorites': tweet_favs,
            'date': tweet_date,
            'language': tweet_lang,
            'user_id': user_api.id_str,
            'user_screen_name': user_api.screen_name
        }
        tweets.append(tweet)
    df_tweets = df_tweets.append(tweets, ignore_index=True)

In [ ]:
def extract_depression_user(depression_users):
    users = []

    for u in depression_users:
        user = dict()
        screen_name = u
        user_api = api.get_user(screen_name=screen_name)
        user = {
            'id': user_api.id_str,
            'name': user_api.name,
            'screen_name': user_api.screen_name,
            'location': user_api.location,
            'number_of_followers': user_api.followers_count,
            'number_of_friends': user_api.friends_count,
            'number_of_tweets': user_api.statuses_count,
            'created_at': user_api.created_at
        }
        users.append(user)
    return users

In [ ]:
def extract_data(depression_user):
    user = extract_depression_user(depression_user)
    return user

In [ ]:
depression_users = users_twitter.depression_users
depression_data = extract_data(depression_users)

In [ ]:
non_depression_users = users_twitter.non_depression_users
non_depression_data = extract_data(non_depression_users)

In [ ]:
df_depression_users = df_depression_users.append(pd.DataFrame(depression_data), sort=False)
df_non_depression_users = df_non_depression_users.append(pd.DataFrame(non_depression_data), sort=False)

In [ ]:
df_depression_users['depression'] = True
df_non_depression_users['depression'] = False

In [ ]:
len(df_depression_users)

In [ ]:
len(df_non_depression_users)

In [ ]:
df_all_users = (
    df_depression_users
    .append(df_non_depression_users)
)

In [ ]:
df_all_users.head()

In [ ]:
# df_all_users.to_csv('all_users.csv', index=False)